In [3]:
import pandas as pd
import numpy as np
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.preprocessing import LabelEncoder
import warnings
warnings.filterwarnings('ignore')

In [5]:
pizza_df = pd.read_csv('pizza_sales.csv')
ingred_df = pd.read_csv('Pizza_ingredients.csv')

In [7]:
pizza_df.dropna(inplace=True)

In [9]:
def parse_dates(date):
  for fmt in ('%d-%m-%Y', '%d/%m/%Y'):
    try:
      return pd.to_datetime(date, format=fmt)
    except ValueError:
      pass
  raise ValueError(f'no valid date format found for {date}')

In [11]:
pizza_df['order_date'] = pizza_df['order_date'].apply(parse_dates)

In [15]:
pizza_df= pizza_df[['order_date', 'pizza_name', 'quantity']]

In [17]:
sales_summary = pizza_df.groupby(['order_date', 'pizza_name']).sum().reset_index()

In [19]:
sales_summary.head()

,order_date,pizza_name,quantity
0,2015-01-01,The Barbecue Chicken Pizza,11
1,2015-01-01,The Big Meat Pizza,5
2,2015-01-01,The Calabrese Pizza,1
3,2015-01-01,The California Chicken Pizza,4
4,2015-01-01,The Chicken Alfredo Pizza,2


In [21]:
# reshape data for time series modeling
sales_pivot = sales_summary.pivot(index='order_date', columns='pizza_name', values='quantity')

In [23]:
sales_pivot

pizza_name,The Barbecue Chicken Pizza,The Big Meat Pizza,The Brie Carre Pizza,The Calabrese Pizza,The California Chicken Pizza,The Chicken Alfredo Pizza,The Chicken Pesto Pizza,The Classic Deluxe Pizza,The Five Cheese Pizza,The Four Cheese Pizza,...,The Prosciutto and Arugula Pizza,The Sicilian Pizza,The Soppressata Pizza,The Southwest Chicken Pizza,The Spicy Italian Pizza,The Spinach Pesto Pizza,The Spinach Supreme Pizza,The Spinach and Feta Pizza,The Thai Chicken Pizza,The Vegetables + Vegetables Pizza
order_date,,,,,,,,,,,,,,,,,,,,,
2015-01-01,11.0,5.0,NaN,1.0,4.0,2.0,4.0,10.0,6.0,5.0,...,4.0,4.0,NaN,7.0,5.0,2.0,5.0,2.0,7.0,3.0
2015-01-02,8.0,8.0,NaN,NaN,5.0,1.0,2.0,6.0,6.0,3.0,...,4.0,6.0,4.0,6.0,11.0,2.0,7.0,4.0,9.0,5.0
2015-01-03,7.0,8.0,NaN,1.0,11.0,5.0,3.0,5.0,4.0,7.0,...,7.0,6.0,1.0,5.0,4.0,3.0,2.0,2.0,9.0,7.0
2015-01-04,9.0,NaN,2.0,2.0,5.0,4.0,1.0,4.0,2.0,4.0,...,4.0,3.0,NaN,1.0,4.0,3.0,4.0,3.0,5.0,1.0
2015-01-05,2.0,3.0,2.0,NaN,6.0,8.0,2.0,9.0,5.0,4.0,...,2.0,5.0,2.0,3.0,2.0,4.0,2.0,2.0,6.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2015-12-27,4.0,5.0,1.0,1.0,3.0,NaN,3.0,8.0,2.0,4.0,...,1.0,4.0,1.0,2.0,4.0,3.0,1.0,1.0,5.0,9.0
2015-12-28,2.0,3.0,1.0,1.0,5.0,2.0,3.0,9.0,2.0,4.0,...,5.0,4.0,5.0,3.0,2.0,2.0,2.0,3.0,6.0,3.0
2015-12-29,4.0,6.0,1.0,2.0,6.0,1.0,1.0,3.0,3.0,3.0,...,3.0,2.0,3.0,1.0,3.0,4.0,NaN,1.0,5.0,1.0


In [25]:
arima_models = {}

for pizza_name in sales_pivot.columns:
  try:
    model = ARIMA(sales_pivot[pizza_name], order=(1, 1, 0))
    model_fit = model.fit()
    arima_models[pizza_name] = model_fit
  except:
    print(f'ARIMA model for {pizza_name} failed to fit')

In [27]:
# Generate predictions for one week
prediction_days = 7
predictions_arima = {}

for pizza_name, model in arima_models.items():
  predictions_arima[pizza_name] = model.predict(start=len(sales_pivot), end=len(sales_pivot) + prediction_days - 1)
     

In [29]:
predictions_df = pd.DataFrame(predictions_arima)

In [31]:
predictions_df.index = pd.date_range(start=sales_pivot.index[-1], periods=prediction_days, freq='D')

In [33]:
ingredients_df = ingred_df[['pizza_name', 'pizza_ingredients', 'Items_Qty_In_Grams']]

In [43]:
# change column name to items_qty
ingredients_df.rename(columns={'Items_Qty_In_Grams': 'items_qty'}, inplace=True)

In [45]:
predictions_df

,The Barbecue Chicken Pizza,The Big Meat Pizza,The Brie Carre Pizza,The Calabrese Pizza,The California Chicken Pizza,The Chicken Alfredo Pizza,The Chicken Pesto Pizza,The Classic Deluxe Pizza,The Five Cheese Pizza,The Four Cheese Pizza,...,The Prosciutto and Arugula Pizza,The Sicilian Pizza,The Soppressata Pizza,The Southwest Chicken Pizza,The Spicy Italian Pizza,The Spinach Pesto Pizza,The Spinach Supreme Pizza,The Spinach and Feta Pizza,The Thai Chicken Pizza,The Vegetables + Vegetables Pizza
2015-12-31,7.949231,4.406059,1.635145,2.305404,7.0,2.507159,3.501351,6.906835,3.136877,6.979677,...,3.623179,4.505280,2.025671,5.914767,5.817878,3.424919,2.0,3.166731,3.0,7.853792
2016-01-01,9.000644,5.252942,1.843755,2.719451,7.0,3.250017,3.749554,8.501447,4.004684,8.000103,...,4.255058,4.748308,2.500329,5.496368,7.264435,4.251879,2.0,4.006950,3.0,8.775034
2016-01-02,8.461593,4.802982,1.724480,2.472638,7.0,2.880361,3.626011,7.679380,3.600476,7.484705,...,3.965063,4.628922,2.269092,5.687737,6.606846,3.817702,2.0,3.621863,3.0,8.379599
2016-01-03,8.737960,5.042051,1.792677,2.619763,7.0,3.064307,3.687505,8.103178,3.788748,7.745023,...,4.098153,4.687569,2.381743,5.600208,6.905779,4.045657,2.0,3.798355,3.0,8.549336
2016-01-04,8.596269,4.915030,1.753684,2.532062,7.0,2.972773,3.656896,7.884699,3.701055,7.613542,...,4.037073,4.658759,2.326864,5.640242,6.769887,3.925975,2.0,3.717466,3.0,8.476478
2016-01-05,8.668913,4.982518,1.775979,2.584340,7.0,3.018321,3.672131,7.997331,3.741901,7.679950,...,4.065105,4.672912,2.353599,5.621931,6.831662,3.988811,2.0,3.754539,3.0,8.507751
2016-01-06,8.631669,4.946661,1.763232,2.553177,7.0,2.995656,3.664548,7.939266,3.722875,7.646408,...,4.052240,4.665960,2.340574,5.630306,6.803580,3.955820,2.0,3.737547,3.0,8.494327


In [47]:
ingredients_df.head()

,pizza_name,pizza_ingredients,items_qty
0,The Barbecue Chicken Pizza,Barbecued Chicken,40.0
1,The Barbecue Chicken Pizza,Red Peppers,15.0
2,The Barbecue Chicken Pizza,Green Peppers,20.0
3,The Barbecue Chicken Pizza,Tomatoes,30.0
4,The Barbecue Chicken Pizza,Red Onions,60.0


In [49]:
# Create a dictionary to store the ingredient quantities
ingredient_quantities = {}

# Iterate through each pizza in the predictions
for pizza_name in predictions_df.columns:
  # Get the predicted quantity for the pizza
  predicted_quantity = predictions_df[pizza_name].sum()

  # Get the ingredients for the pizza
  pizza_ingredients = ingredients_df[ingredients_df['pizza_name'] == pizza_name]

  # Iterate through each ingredient for the pizza
  for index, row in pizza_ingredients.iterrows():
    ingredient = row['pizza_ingredients']
    ingredient_qty = row['items_qty']

    # Calculate the required quantity of the ingredient
    required_quantity = predicted_quantity * ingredient_qty

    # Add the required quantity to the dictionary
    if ingredient not in ingredient_quantities:
      ingredient_quantities[ingredient] = 0
    ingredient_quantities[ingredient] += required_quantity


In [51]:
# Create a DataFrame from the ingredient quantities
ingredient_requirements_df = pd.DataFrame.from_dict(ingredient_quantities, orient='index', columns=['required_quantity'])

In [53]:
ingredient_requirements_df

,required_quantity
Barbecued Chicken,5404.165209
Red Peppers,11546.278091
Green Peppers,8056.630737
Tomatoes,36210.706643
Red Onions,55348.994016
...,...
Luganega Sausage,3908.125169
Onions,1954.062585
Soppressata Salami,4859.361666
Peperoncini verdi,1410.002025


In [55]:
# Create a purchase order DataFrame
purchase_order_df = ingredient_requirements_df.copy()

# Add a column for the unit of measure (assuming all ingredients are in grams)
purchase_order_df['unit'] = 'grams'

# Rename the columns for better readability
purchase_order_df = purchase_order_df.rename(columns={'required_quantity': 'quantity'})

# Add a header to the purchase order
print('Purchase Order:')
print('----------------')

# Print the purchase order table
print(purchase_order_df.to_string())

Purchase Order:
----------------
                                quantity   unit
Barbecued Chicken            5404.165209  grams
Red Peppers                 11546.278091  grams
Green Peppers                8056.630737  grams
Tomatoes                    36210.706643  grams
Red Onions                  55348.994016  grams
Barbecue Sauce               1801.388403  grams
Bacon                       20005.513107  grams
Pepperoni                   24219.867700  grams
Italian Sausage               343.482441  grams
Chorizo Sausage              1717.412207  grams
Brie Carre Cheese             368.668512  grams
Prosciutto                    368.668512  grams
Caramelized Onions                   NaN  grams
Pears                         122.889504  grams
Thyme                          61.444752  grams
Garlic                      18571.887062  grams
?duja Salami                 2134.420269  grams
Pancetta                     3201.630404  grams
Friggitello Peppers           533.605067  grams
Chicken